In [2]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [3]:
from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
import seaborn as sns
import random
from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import *

In [14]:
#define a function to get IB data. endDate is the last date of the historical data 

def get_data(contract, history, freq, side, endDate =''):
    bar = ib.reqHistoricalData(
        contract,
        endDateTime = endDate,
        durationStr=history,
        barSizeSetting= freq,
        whatToShow=side,
        useRTH=True,
        formatDate=1)
    return util.df(bar)

history = '6 Y'
freq = '1 day'
side ='BID_ASK'

In [ ]:
# Pull data from ADRs in "Asian_ADRs.csv"
adr_symbol = "VEDL"
adr_exchange = 'SMART'
adr_currency = "USD"

underlying_symbol = 'VEDL'
underlying_exchange = 'NSE'
underlying_currency = 'INR'
underlying_country = "India"

folder_name = underlying_country + "/" + adr_symbol + "_" + underlying_symbol + "/"

In [ ]:
adr_contract = Contract(symbol = adr_symbol, secType = 'STK', exchange = adr_exchange, currency = adr_currency)
ib.qualifyContracts(adr_contract)
adr_df = get_data(adr_contract, history, freq, side)
print(adr_df.head(3))
adr_df.to_csv(folder_name + "adr.csv")

In [ ]:
underlying_contract = Contract(symbol = underlying_symbol, secType = 'STK', exchange = underlying_exchange, currency = underlying_currency)
ib.qualifyContracts(underlying_contract)
underlying_df = get_data(underlying_contract, history, freq, side)
print(underlying_df.head(3))
underlying_df.to_csv(folder_name + "underlying.csv")

In [1]:
# No underlying data for:
# VEDL-VEDL
# PLL-PLL
# IMMP-IMM
# KZIA-KZA

In [47]:
currency1 = "USD"
secType = "CASH"
currency2 = "RUB"
exchange = "IDEALPRO"

folder_name = "Forex/"

In [48]:
forex_contract = Contract(symbol = currency1, secType = secType, exchange = exchange, currency = currency2)
ib.qualifyContracts(forex_contract)
forex_df = get_data(forex_contract, history, freq, side)
print(forex_df.head(3))
forex_df.to_csv(folder_name + currency1 + "_" + currency2 + ".csv")

         date      open      high       low     close  volume  average  \
0  2015-04-10  51.95738  53.73462  50.25350  52.15030      -1     -1.0   
1  2015-04-13  52.97532  55.05547  51.84043  53.09138      -1     -1.0   
2  2015-04-14  51.65213  52.89562  50.61336  51.79561      -1     -1.0   

   barCount  
0        -1  
1        -1  
2        -1  
